In [9]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
splitedRdd = sc.parallelize(splitedRdd.take(20000))

#splitedRdd.take(1)
#(uid,[[train],[test]]

In [10]:
ContextRow2 = namedtuple("context", ["ts","city", "lat", "lon", "moving", "location", "time_of_day",
                                     "last_app", "app_chain" ])
def addLastApp(row):
    train = row[1][0]
    test = row[1][1]
    if (len(train) == 0 or len(test) == 0):
        #cannot evaluate
        return (row[0], [[],[]])
    
    temp_list = list(train[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(-1) #last app unknown
    temp_list[1].append(-1) #app chain unknown
    new_row = [[TrainRow(temp_list[0],ContextRow2(*temp_list[1]))]]
    
    ts = test[0].context.ts - train[-1].context.ts
    app_chain_temp = -1 if ts > 60 else train[-1].itemId
    temp_list = list(test[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(train[-1].itemId) #last app 
    temp_list[1].append(app_chain_temp) #app chain
    new_row.append([TrainRow(temp_list[0],ContextRow2(*temp_list[1]))])
    for i in xrange(1,len(train)):
        ts = train[i].context.ts - train[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else train[i-1].itemId
        temp_list = list(train[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(train[i-1].itemId) #last app 
        temp_list[1].append(app_chain_temp) #app chain 
        new_row[0].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    for i in xrange(1,len(test)):
        ts = test[i].context.ts - test[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else test[i-1].itemId
        temp_list = list(test[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(test[i-1].itemId) #last app 
        temp_list[1].append(app_chain_temp) #app chain 
        new_row[1].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    return (row[0], new_row)
        
splitedRddv2 = splitedRdd.map(addLastApp)
#splitedRddv2.take(1)

In [11]:
splitedRdd_flat = splitedRddv2.flatMap(lambda x: x[1]).flatMap(lambda x: x)
splitedRdd_flat.take(2)

[train(itemId=94874, context=context(ts=1420959850, city=2995, lat=42.897598, lon=-87.853897, moving=1, location=1, time_of_day=1, last_app=-1, app_chain=-1)),
 train(itemId=14861, context=context(ts=1420982193, city=2995, lat=42.897644, lon=-87.853874, moving=1, location=2, time_of_day=2, last_app=94874, app_chain=-1))]

In [12]:
numRecList = 5
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores

def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation2.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()

In [13]:
index_of_context = 8
numAll = splitedRdd_flat.count() 
#p_context giving me co8ntext giving back p
p_all_context_list = splitedRdd_flat.map(lambda x: x[1][index_of_context]).map(lambda x:(x,1.0)).groupByKey().map(lambda x:(x[0],sum(x[1])/numAll)).collect()
#p_all_context_list.take(10)
#p_all_context_list[[x[0] for x in p_all_context_list].index(72320)][1]

In [ ]:
#p_appi giving me appi giving back p
p_all_appid_list = splitedRdd_flat.map(lambda x: x[0]).map(lambda x:(x,1.0)).groupByKey().map(lambda x:(x[0],sum(x[1])/numAll)).collect()
#all_appid.take(10)
#p_all_appid_list[[x[0] for x in p_all_appid_list].index(77952)][1]

In [ ]:
index_of_context = 8
#all_data = splitedRdd_flat.map(lambda x: (x[0],x[1][index_of_context]))
#all_data.take(10)
all_appid_list = splitedRdd_flat.map(lambda x: x[0]).distinct().collect()
#all_appid_list.take(10)
all_data_list = splitedRdd_flat.map(lambda x: (x[0],x[1][index_of_context])).collect()
context_model = splitedRdd_flat.map(lambda x: x[1][index_of_context]).distinct()

def bayesian_model(line):
    context_id = line
    p_context = p_all_context_list[[x[0] for x in p_all_context_list].index(context_id)][1]
    p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
    for appid in all_appid_list:
        num_appid = 0
        num_appid_context = 0
        for i in all_data_list:
            if i[0] == appid:
                num_appid = num_appid + 1.0
                if i[1] == context_id:
                    num_appid_context  = num_appid_context + 1.0  
        if num_appid != 0:
            p_contextAppid = num_appid_context/num_appid
        else:
            p_contextAppid = 0
        
        p_appi =  p_all_appid_list[[x[0] for x in p_all_appid_list].index(appid)][1]
        if p_context != 0:
            p = p_contextAppid * p_appi / p_context
        else:
            p = 0
        p_app = topFiveSortedList(p_app,appid,p)    
    return line,p_app

context_model = context_model.map(bayesian_model)
context_model_list= context_model.collect()
#context_model_list[[x[0] for x in context_model_list].index(72320)][1]


In [ ]:
index_of_context = 8
def bayesian_context(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][index_of_context]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][index_of_context]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = testList[0][1]    
        if context in [x[0] for x in context_model_list]:
            p_app = context_model_list[[x[0] for x in context_model_list].index(context)][1]  
        else:
            p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
            
        testList_items = [t[0] for t in testList]               #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for test set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        #trainList.append(testList[0])
        testList.pop(0)       

    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    #return p_app
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context)
#finalScore.take(20)
print_result(finalScore, 'bayesian_context')

In [64]:
a = [(1,[3,3]),(2,[4,4])]
b = a[[x[0] for x in a].index(1)][1]
if 1 in [x[0] for x in a]:
    print "asdf"


asdf


In [77]:
a = [1,2,3]
1 in a

True